In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project 

/content/drive/MyDrive/Project


In [3]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


In [4]:
! pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00


In [5]:
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, MaxPool2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [6]:
# Creating dataframe with art movement class weights
art_movements = pd.DataFrame()
art_movements['names'] = ['Academic', 'Baroque', 'Expressionism', 'Japanese', 'Neoclassicism', 'Nouveau', 'Primitivism', 'Realism', 'Renaissance', 'Rococo','Romanticism', 'Symbolism', 'Western Medieval']
art_movements['# of paintings'] = [500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]
art_movements = art_movements.sort_values(by=['# of paintings'], ascending=False)
art_movements = art_movements.reset_index(drop=True)
art_movements['class_weight'] = art_movements['# of paintings'].sum() / (art_movements.shape[0] * art_movements['# of paintings'])
class_weights = art_movements['class_weight'].to_dict()
print(class_weights)
art_movements_name = art_movements['names'].values
print(art_movements_name)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0}
['Academic' 'Baroque' 'Expressionism' 'Japanese' 'Neoclassicism' 'Nouveau'
 'Primitivism' 'Realism' 'Renaissance' 'Rococo' 'Romanticism' 'Symbolism'
 'Western Medieval']


In [7]:
batch_size = 64
train_input_shape = (256, 256, 3)

datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range= 5,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Project/new_dataset',
    target_size=train_input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle =True,
    subset='training',
    seed=123,
    classes=art_movements_name.tolist()
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Project/new_dataset', #use the same data with train
    target_size=train_input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle =True,
    subset='validation',
    seed=123,
    classes=art_movements_name.tolist()
)

Found 1200 images belonging to 13 classes.
Found 133 images belonging to 13 classes.


In [8]:
# Adding model steps
Step_Train = train_generator.n//train_generator.batch_size
Step_Validation = validation_generator.n//validation_generator.batch_size
print("Total number of batches =", Step_Train, "and", Step_Validation)

Total number of batches = 18 and 2


In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt
import glob 
import os 
import cv2
import math
from keras import applications
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Convolution2D,Activation,Flatten,Dense,Dropout,MaxPool2D,BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [11]:
# Convolution Neural Networks (CNN)
model = Sequential()

model.add(Conv2D(8, kernel_size=(3,3), padding='same', input_shape = (256,256,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Conv2D(16, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(13, activation='softmax'))
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 8)       224       
                                                                 
 activation (Activation)     (None, 256, 256, 8)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 16)        1168      
                                                                 
 batch_normalization (BatchN  (None, 85, 85, 16)       64        
 ormalization)                                                   
                                                                 
 activation_1 (Activation)   (None, 85, 85, 16)        0

In [12]:
model.fit(train_generator, 
          validation_data=validation_generator, 
          epochs=50)

Epoch 1/50
19/19 [==============================] - 292s 15s/step - loss: 3.1565 - accuracy: 0.0967 - val_loss: 2.5569 - val_accuracy: 0.1278
Epoch 2/50
19/19 [==============================] - 78s 4s/step - loss: 3.1029 - accuracy: 0.1017 - val_loss: 2.5705 - val_accuracy: 0.0977
Epoch 3/50
19/19 [==============================] - 83s 4s/step - loss: 3.0718 - accuracy: 0.1050 - val_loss: 2.5711 - val_accuracy: 0.0977
Epoch 4/50
19/19 [==============================] - 81s 4s/step - loss: 3.0504 - accuracy: 0.0983 - val_loss: 2.5810 - val_accuracy: 0.0752
Epoch 5/50
19/19 [==============================] - 84s 4s/step - loss: 3.0035 - accuracy: 0.1092 - val_loss: 2.5867 - val_accuracy: 0.1128
Epoch 6/50
19/19 [==============================] - 79s 4s/step - loss: 2.9935 - accuracy: 0.1158 - val_loss: 2.5961 - val_accuracy: 0.0977
Epoch 7/50
19/19 [==============================] - 79s 4s/step - loss: 3.0008 - accuracy: 0.1058 - val_loss: 2.5986 - val_accuracy: 0.0752
Epoch 8/50
19/19 [